In [11]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [12]:
def load_data(data_dir, img_size=(224, 224)):
    """
    Load dataset from directory.

    Args:
    - data_dir (str): Path to the directory containing the dataset.
    - img_size (tuple): Size to resize the images.

    Returns:
    - images (list): List of resized images.
    - labels (list): List of corresponding labels.
    """
    images = []
    labels = []
    for label, category in enumerate(['Closed_Eyes', 'Open_Eyes']):
        category_dir = os.path.join(data_dir, category)
        for img_name in os.listdir(category_dir):
            img_path = os.path.join(category_dir, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

In [13]:
def prepare_data(data_dir, test_size=0.2, img_size=(224, 224)):
    """
    Prepare dataset for training and testing.

    Args:
    - data_dir (str): Path to the directory containing the dataset.
    - test_size (float): Percentage of the dataset to use for testing.
    - img_size (tuple): Size to resize the images.

    Returns:
    - train_set (tuple): Tuple containing train images and labels.
    - test_set (tuple): Tuple containing test images and labels.
    """
    # 데이터 로드
    images, labels = load_data(data_dir, img_size)
    
    # 훈련 데이터, 테스트 데이터 분리
    train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # 이미지 정규화
    train_images = train_images.astype('float32') / 255.0
    test_images = test_images.astype('float32') / 255.0
    
    return (train_images, train_labels), (test_images, test_labels)

In [14]:
# 데이터셋 제작
(train_images, train_labels), (test_images, test_labels) = prepare_data(r'C:\Users\impra\Desktop\2024_first_semester\AIUI\Stracker\eye-detection-model\jiwon\drowsy_dataset')

In [15]:
# Resnet50v2 모델 가지고 모델 제작
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# ResNet50V2의 마지막 레이어 직전까지 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 구성 수정
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)  # 가중치 규제 적용
x = Dropout(0.3)(x)  # 드롭아웃 추가
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.00008), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(train_images, train_labels, batch_size=16, epochs=2, validation_data=(test_images, test_labels))

# 모델 실행
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/2
918/918 ━━━━━━━━━━━━━━━━━━━━ 532s 565ms/step - accuracy: 0.9393 - loss: 2.7588 - val_accuracy: 0.9986 - val_loss: 0.3968
Epoch 2/2
918/918 ━━━━━━━━━━━━━━━━━━━━ 489s 533ms/step - accuracy: 0.9974 - loss: 0.2964 - val_accuracy: 0.9986 - val_loss: 0.1215
115/115 ━━━━━━━━━━━━━━━━━━━━ 105s 914ms/step - accuracy: 0.9982 - loss: 0.1213
Test Loss: 0.12145296484231949, Test Accuracy: 0.9986376166343689


In [16]:
# 네이티브 Keras 형식으로 모델 저장
model.save('Improved_Resnet50v2_model.keras')